In [5]:
!pip install "transformers==4.52.3" accelerate sentencepiece safetensors
!pip install qwen-omni-utils[decord]

  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached sentencepiece-0.2.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached safetensors-0.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2025.11.3-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached torch-2.9.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_

In [6]:
!git clone https://github.com/QwenLM/Qwen2.5-Omni.git

fatal: destination path 'Qwen2.5-Omni' already exists and is not an empty directory.


In [7]:
!pip install "git+https://github.com/huggingface/transformers@v4.51.3-Qwen2.5-Omni-preview"
!pip install accelerate

  Cloning https://github.com/huggingface/transformers (to revision v4.51.3-Qwen2.5-Omni-preview) to /tmp/pip-req-build-p5dizy5i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-p5dizy5i
  Running command git checkout -q f551466201fb4089d6df9dc55d80f0edbd149d85
  Resolved https://github.com/huggingface/transformers to commit f551466201fb4089d6df9dc55d80f0edbd149d85
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11252795 sha256=0031b99b613ec3efcc76e50ed55e270570b9fce42596823ec687db0a048a6007
  Stored in directory: /tmp/pip-ephem-wheel-cache-x5wjrf18/wheels/9d/a4/d1/9bf234fcabd7d01ba2e55b65f426d7b52be31a63631cd091eb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninst

In [14]:
import torch
from transformers import Qwen2_5OmniForConditionalGeneration, AutoProcessor

print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "CPU")

model_name = "Qwen/Qwen2.5-Omni-7B"

model = Qwen2_5OmniForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,   # good for L40
    device_map="auto",            # let HF spread it on your GPU
)

processor = AutoProcessor.from_pretrained(model_name)

print("Model and processor loaded.")

CUDA available: True
Device count: 1
Current device: 0


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model and processor loaded.


In [15]:
import torch

# EXACT default system prompt (so we stop seeing that warning)
conversation = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."
            }
        ],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Say hello in one short sentence."
            }
        ],
    },
]

# Batch with ONE conversation
conversations = [conversation]

# This follows the Hugging Face Qwen2.5-Omni docs
inputs = processor.apply_chat_template(
    conversations,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,      # 👈 important: returns a dict / BatchEncoding, not a tensor
    return_tensors="pt",
    padding=True,
).to(model.device)

print("Type of inputs:", type(inputs))

with torch.no_grad():
    text_ids = model.generate(
        **inputs,
        return_audio=False,    # text only
        max_new_tokens=64,
    )

# Decode the whole sequence (no manual slicing to keep it simple)
texts = processor.batch_decode(
    text_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False,
)

print("\nModel reply:\n", texts[0])

Type of inputs: <class 'transformers.feature_extraction_utils.BatchFeature'>

Model reply:
 system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.
user
Say hello in one short sentence.
assistant
Hello. How can I assist you today?


In [16]:
!pip install -U qwen-omni-utils soundfile


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
!pip install torchvision
import time
import torchvision
import soundfile as sf
from qwen_omni_utils import process_mm_info

# EXACT system prompt Qwen expects for audio
QWEN_SYSTEM_PROMPT = (
    "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, "
    "capable of perceiving auditory and visual inputs, as well as generating text and speech."
)

def run_qwen_audio_turn(audio_path, speaker="Chelsie", save_path="qwen_reply.wav"):
    """
    audio_path: path to your input WAV file (one of your prompts)
    speaker: "Chelsie" or "Ethan" (built-in voices)
    save_path: where to save Qwen's spoken reply
    """
    # 1) Build conversation with required system prompt + user audio
    conversation = [
        {
            "role": "system",
            "content": [
                {"type": "text", "text": QWEN_SYSTEM_PROMPT}
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "audio", "audio": audio_path},
            ],
        },
    ]

    conversations = [conversation]
    USE_AUDIO_IN_VIDEO = False  # we are not using video here

    # 2) Prepare text template (not tokenized, Qwen will handle multimodal tokens)
    text = processor.apply_chat_template(
        conversations,
        add_generation_prompt=True,
        tokenize=False,
    )

    # 3) Extract multimodal info (audio paths -> tensors)
    audios, images, videos = process_mm_info(
        conversations,
        use_audio_in_video=USE_AUDIO_IN_VIDEO,
    )

    # 4) Pack everything into model inputs
    inputs = processor(
        text=text,
        audio=audios,
        images=images,
        videos=videos,
        return_tensors="pt",
        padding=True,
        use_audio_in_video=USE_AUDIO_IN_VIDEO,
    )
    inputs = inputs.to(model.device).to(model.dtype)

    # 5) Generate text + audio and time it for latency
    t_start = time.time()
    text_ids, audio = model.generate(
        **inputs,
        use_audio_in_video=USE_AUDIO_IN_VIDEO,
        speaker=speaker,      # "Chelsie" or "Ethan"
        max_new_tokens=128,
        return_audio=True,
    )
    t_end = time.time()
    latency = t_end - t_start

    # 6) Decode text
    reply_text = processor.batch_decode(
        text_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False,
    )[0]

    print("Qwen text reply:\n", reply_text)
    print(f"\nEnd-to-end latency: {latency:.3f} seconds")

    # 7) Save reply audio for MOS / naturalness rating
    audio_np = audio.reshape(-1).detach().cpu().numpy()
    sf.write(save_path, audio_np, samplerate=24000)
    print("Saved reply audio to:", save_path)

    return latency, reply_text

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
# Install dependencies
!pip install -q gTTS pydub

from gtts import gTTS
from pydub import AudioSegment

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
# Generate TTS audio and save as mp3
tts = gTTS("Say this sentence in a angry tone: ‘Are you mad?", lang="en")
tts.save("prosody.mp3")

# Convert mp3 to wav
sound = AudioSegment.from_mp3("prosody.mp3")
sound.export("prosody.wav", format="wav")

print("✅ prosody.wav created!")

✅ prosody.wav created!


In [ ]:
prompt_files = [
    "prosody.wav"
]

latencies = []

for i, wav in enumerate(prompt_files, start=1):
    print(f"\n=== Running Qwen on {wav} (Prompt {i}) ===")
    lat, _ = run_qwen_audio_turn(
        wav,
        speaker="Chelsie",
        save_path=f"qwen_reply_prompt{i}.wav",
    )
    latencies.append(lat)

print("\nAll latencies:", latencies)
print("Average latency:", sum(latencies) / len(latencies))


=== Running Qwen on prosody.wav (Prompt 1) ===


Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


In [12]:
prompt_files = [
    "p1.wav",
    "p2.wav",
    "p3.wav",
    "p4.wav",
    "p5.wav",
    "p6.wav",
    "p7.wav",
    "p8.wav",
    "p9.wav",
    "p10.wav",
]

latencies = []

for i, wav in enumerate(prompt_files, start=1):
    print(f"\n=== Running Qwen on {wav} (Prompt {i}) ===")
    lat, _ = run_qwen_audio_turn(
        wav,
        speaker="Chelsie",
        save_path=f"qwen_reply_prompt{i}.wav",
    )
    latencies.append(lat)

print("\nAll latencies:", latencies)
print("Average latency:", sum(latencies) / len(latencies))



=== Running Qwen on p1.wav (Prompt 1) ===


Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


Qwen text reply:
 system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.
user

assistant
Hello! How can I assist you today?

End-to-end latency: 3.629 seconds
Saved reply audio to: qwen_reply_prompt1.wav

=== Running Qwen on p2.wav (Prompt 2) ===


Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


Qwen text reply:
 system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.
user

assistant
Well, 2 plus 2 is 4. It's a simple math fact. If you have any other other math questions or other anything else you want to talk about, feel free to let me know.

End-to-end latency: 15.122 seconds
Saved reply audio to: qwen_reply_prompt2.wav

=== Running Qwen on p3.wav (Prompt 3) ===


...

Qwen text reply:
 system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.
user

assistant
Well, first off, wind energy is renewable. You停顿，嗯，就是说它不会像石油，煤炭那样用完就没了。第二呢，它对环境很友好。风能发电不会产生温室气体排放，像二氧化碳啥的，对保护环境很有帮助。还有啊，风能发电成本在不断降低。随着技术的发展，建风力发电场的成本越来越低了。你要是还有啥想知道的，尽管再问哈。

End-to-end latency: 51.917 seconds
Saved reply audio to: qwen_reply_prompt8.wav

=== Running Qwen on p9.wav (Prompt 9) ===


Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


Qwen text reply:
 system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.
user

assistant
Well, it's about a human named Jake who goes to a planet called Pandora. He's there to collect resources but ends up befriending the Na'vi, the native people there. He learns about their culture and becomes a part of it. In the end, he has to fight to protect the Na'vi and their home planet. So, what do you think about that? Do you have any other other questions about the movie?

End-to-end latency: 29.536 seconds
Saved reply audio to: qwen_reply_prompt9.wav

=== Running Qwen on p10.wav (Prompt 10) ===


Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


Qwen text reply:
 system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.
user

assistant
Well, you know, machine learning is like teaching a computer to learn from data. It has a bunch of data, like pictures or numbers. The computer looks at this data and finds patterns. For example, if a bunch of pictures of cats and dogs. It can learn what makes a picture a cat or a dog. Then, when it sees a new picture, it can figure out if it's a cat or a dog based on what it learned before. It doesn't need to be told exactly what to do, it just figures it out from the data. So, it's kind of like a computer learning to play a game by watching other play it over times.If you want to know more about it, like how it actually does the learning part, just let me know.

End-to-end latency: 70.639 seconds
Saved reply audio to: qwen_reply_prompt10.wav

All latencies: [3.629286289215088, 15.1221